# 09 - Resultados Finales para la Tesis

## Objetivo
Exportar:
- Gráficos
- Tablas resumen
- Resultados clave

## Formato
Se generan imágenes para la tesis y tablas en LaTeX o CSV


In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(time, velocity)
plt.title("Velocidad de eritrocitos a lo largo del tiempo")
plt.xlabel("Tiempo (s)")
plt.ylabel("Velocidad (μm/s)")
plt.savefig("figures/velocity_curve.png")
